In [2]:
### Convert RAW DICOM data to NIFTI Image Data set
import os, sys
from os.path import join as oj
from glob import glob

In [13]:
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from nipype.interfaces.dcm2nii import Dcm2nii
from nipype.caching import Memory

mem = Memory(base_dir=NiPypeOutputDir)  ## Create a memory cache I can use going forward


In [4]:
## This is the location of the Raw DICOM Files from BITC
StoutRawData = '/home/ehecht/BIGDATA/Stout_Homo_Faber/RAWDATA'
NiPypeOutputDir ='/EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/'

In [14]:
## Figure out which directories I want to run through the preprocessing pipeline
HomoFaberImageSessions = [ x for x in os.listdir(StoutRawData) if x.startswith('HOMO')]

print len(HomoFaberImageSessions),"Image Sets Found for HomoFaber"

113 Image Sets Found for HomoFaber


In [16]:
### gnerate my list of directories to process

T1ImageInputDirectories =  glob( oj(StoutRawData,'*/*diff_AP*'))

### So converting this back into a memcached version
dcmConverter = mem.cache(Dcm2nii)



In [8]:
stoutDCM2NII_wf = pe.Workflow('stoutDCM2NII_wf') ## Initialize the workflow
stoutDCM2NII_wf.base_dir = NiPypeOutputDir  ## Tell it where to dump the results of the workflow

### Create the list of Subjects to Process
# imageSession_Id_InfoSrc  = pe.Node(util.IdentityInterface(fields=['imageSessionName']),name='imageSession_Id_InfoSrc')
# imageSession_Id_InfoSrc.iterables = ('imageSessionName', HomoFaberImageSessions)
t1ImageDir_InfoSrc  = pe.Node(util.IdentityInterface(fields=['t1ImageDir']),name='t1ImageDir_InfoSrc')
t1ImageDir_InfoSrc.iterables = ('t1ImageDir', T1ImageInputDirectories)





In [ ]:
# ## Create a datasource.. this basically helps me find the individual image files and data sets for an image session
# ## a single image directory likely consists of DTI data, T2 images, T1 images, etc, etc
# datasource = pe.Node(interface=nio.DataGrabber(infields=['imageSessionName'], outfields=['t1Mprage_dir']), name='datasource')
# datasource.inputs.base_directory = StoutRawData
# datasource.inputs.template = '%s/t1_mprage_sag_*'
# datasource.inputs.sort_filelist = True
# datasource.inputs.template_args = dict( t1Mprage_dir=[['imageSessionName']])
# ## Create a datasource.. this basically helps me find the individual image files and data sets for an image session
# ## a single image directory likely consists of DTI data, T2 images, T1 images, etc, etc
# datasource = pe.Node(interface=nio.DataGrabber(infields=['imageSessionName'], outfields=['t1Mprage_dir']), name='datasource')
# datasource.inputs.base_directory = StoutRawData
# datasource.inputs.template = '%s/t1_mprage_sag_*'
# datasource.inputs.sort_filelist = True
# datasource.inputs.template_args = dict( t1Mprage_dir=[['imageSessionName']])

In [9]:
## Now create a node for the dicom converter!
dcmConvert = pe.Node(interface=Dcm2nii(),name='dcmConvert')
dcmConvert.inputs.gzip_output = False
dcmConvert.inputs.reorient = False
dcmConvert.inputs.reorient_and_crop = False

In [10]:
# stoutDCM2NII_wf.connect(imageSession_Id_InfoSrc, 'imageSessionName', datasource, 'imageSessionName')
# stoutDCM2NII_wf.connect(datasource,'t1Mprage_dir', dcmConvert, 'source_dir' )
stoutDCM2NII_wf.connect(t1ImageDir_InfoSrc, 't1ImageDir', dcmConvert, 'source_dir' )



In [ ]:
stoutDCM2NII_wf.run(plugin='MultiProc', plugin_args={'n_procs' : 8})

161130-22:00:42,318 workflow INFO:
	 ['check', 'execution', 'logging']
161130-22:00:43,522 workflow INFO:
	 Running in parallel.
161130-22:00:43,538 workflow INFO:
	 Executing: dcmConvert.a243 ID: 0
161130-22:00:43,542 workflow INFO:
	 Executing: dcmConvert.a209 ID: 1
161130-22:00:43,546 workflow INFO:
	 Executing: dcmConvert.a476 ID: 2
161130-22:00:43,545 workflow INFO:
	 Executing node dcmConvert.a243 in dir: /EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/stoutDCM2NII_wf/_t1ImageDir_..home..ehecht..BIGDATA..Stout_Homo_Faber..RAWDATA..HOMO_FABERSubject14_1627_BITC20160713..cmrr_mbep2d_diff_AP_14/dcmConvert
161130-22:00:43,550 workflow INFO:
	 Executing: dcmConvert.a498 ID: 3
161130-22:00:43,550 workflow INFO:
	 Executing node dcmConvert.a209 in dir: /EINSTEIN_BIGDATA/NIPYPE_WD/Stout_Haber/stoutDCM2NII_wf/_t1ImageDir_..home..ehecht..BIGDATA..Stout_Homo_Faber..RAWDATA..HOMO_FABERSubject17_1423_BITC20160318..cmrr_mbep2d_diff_AP_FA_16/dcmConvert
161130-22:00:43,555 workflow INFO:
	 Executing: dc